### Twitter Data Analysis


In [1]:
import os
from fnmatch import fnmatch
import pandas as pd
import numpy as np
import nltk
import seaborn
import spacy
import glob
import string
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import datetime
import re

In [2]:
filename=[]
root = '../data/'
pattern = "*.csv"

for path, subdirs, files in os.walk(root):
    for name in files:
        if fnmatch(name, pattern):
            filename.append(os.path.join(path, name))

In [33]:
data = pd.concat( [ pd.read_csv(file) for file in filename ] )
data.reset_index(inplace=True)


In [34]:
data.head()

,index,date,tweet,word
0,0,2019-01-20 23:04:24+00:00,Logitech Prodigy G213 Gaming Keyboard [50% dis...,amazonindia
1,1,2019-01-20 22:47:26+00:00,Can you guys (whoever's reading this) dm @/Ama...,amazonindia
2,2,2019-01-20 22:04:27+00:00,Artis E400M in-Ear Headphones with Mic (Black)...,amazonindia
3,3,2019-01-20 21:24:51+00:00,@JeffBezos called Amazon India to report an in...,amazonindia
4,4,2019-01-20 21:04:30+00:00,TP-Link HA100 NFC-Enabled Bluetooth Music Rece...,amazonindia


In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45144 entries, 0 to 45143
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   45144 non-null  int64 
 1   date    45144 non-null  object
 2   tweet   45143 non-null  object
 3   word    45144 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.4+ MB


In [36]:
data['Date'] = [datetime.datetime.fromisoformat(x).date() for x in data['date']]
data.drop(['date','index','word'],axis=1,inplace=True)

In [37]:
data.head()

,tweet,Date
0,Logitech Prodigy G213 Gaming Keyboard [50% dis...,2019-01-20
1,Can you guys (whoever's reading this) dm @/Ama...,2019-01-20
2,Artis E400M in-Ear Headphones with Mic (Black)...,2019-01-20
3,@JeffBezos called Amazon India to report an in...,2019-01-20
4,TP-Link HA100 NFC-Enabled Bluetooth Music Rece...,2019-01-20


In [38]:
# print (re.findall(r'(?<=\W)[#@]\S*', data['tweet']))

In [39]:
data['tag']=None
data['mention']=None
for i in range(data.shape[0]):
    try:
        data['tag'][i] = re.findall(r'(?<=\W)[#]\S*', data['tweet'][i])
        data['mention'][i] = re.findall(r'(?<=\W)[@]\S*', data['tweet'][i])
        
    except TypeError:
        pass

In [57]:
data

,tweet,Date,tag,mention,is_mention
0,Logitech Prodigy G213 Gaming Keyboard [50% dis...,2019-01-20,"[#AmazonIndia, #DealsOnPC]",[],0
1,Can you guys (whoever's reading this) dm @/Ama...,2019-01-20,[],[@/AmazonHelp],0
2,Artis E400M in-Ear Headphones with Mic (Black)...,2019-01-20,"[#AmazonIndia, #DealsOnElectronics]",[],0
3,@JeffBezos called Amazon India to report an in...,2019-01-20,[],[@AmazonHelp],0
4,TP-Link HA100 NFC-Enabled Bluetooth Music Rece...,2019-01-20,"[#AmazonIndia, #DealsOnPC]",[],0
...,...,...,...,...,...
45139,Murder. Corruption. Romance. Movie stars. A mo...,2019-12-27,"[#ASMSG, #amazonindia]",[@SarahMallery1],0
45140,Iball Cr24 Card Reader With 24 Types Of Memory...,2019-12-27,"[#AmazonIndia, #DealsOnPC]",[],0
45141,Murder is on the itinerary! ★MURDER OVER MIAMI...,2019-12-27,"[#ASMSG, #amazonindia]",[],0
45142,AmazonBasics 9 Volt Everyday Alkaline Batterie...,2019-12-27,"[#AmazonIndia, #DealsOnElectronics]",[],0


In [53]:
len(data['mention'][0])

0

In [ ]:
data['is_mention'] = None

In [56]:
for mention in data['mention']:
    if mention!=None:
        mention1 = [x.lower() for x in mention]
        r = re.compile(".*amazon")
        l = list(filter(r.match, mention1))
        if len(l)>0:
            data['is_mention'] = 1
        else:
            data['is_mention'] = 0

In [63]:
mylist = ["dog", "cat", "wildCat", "thundercat1", "cow", "hooo"]
r = re.compile(".*cat")
list(filter(r.match, mylist)) # Read Note


['cat', 'thundercat1']